In [1]:
import time
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats

In [2]:
POSITION_TYPES = ['CENTER', 'GUARD', 'FORWARD']

# Centers
centers=[
    'nikola jokic',
    'joel embiid',
    'anthony davis',
    'clint capela',
    'bam adebayo'
]
center_ids=[]
for i in range(len(centers)):
    center_ids.append(players.find_players_by_full_name(centers[i])[0]['id'])

# Guards
guards=[
    'damian lillard',
    'stephen curry',
    'luka doncic',
    'jamal murray',
    'james harden'
]
guard_ids=[]
for i in range(len(guards)):
    guard_ids.append(players.find_players_by_full_name(guards[i])[0]['id'])

# Forwards
forwards=[
    'giannis antetokounmpo',
    'draymond green',
    'pascal siakam',
    'zion williamson',
    'blake griffin'
]
forward_ids=[]
for i in range(len(forwards)):
    forward_ids.append(players.find_players_by_full_name(forwards[i])[0]['id'])

In [3]:
nba_players = []
positions = [center_ids, guard_ids, forward_ids]
for position in positions:
    for i in range(len(position)):
        time.sleep(0.600)
        career = playercareerstats.PlayerCareerStats(player_id=position[i])
        nba_players.append(career.get_data_frames()[0])

In [4]:
X_train = np.zeros((15,6))
for i in range(len(nba_players)):
    X_train[i] = np.array([nba_players[i]['PLAYER_ID'][0],
                           (sum(nba_players[i]['PTS']) / sum(nba_players[i]['GP'])), 
                           (sum(nba_players[i]['AST']) / sum(nba_players[i]['GP'])), 
                           (sum(nba_players[i]['REB']) / sum(nba_players[i]['GP'])), 
                           (sum(nba_players[i]['STL']) / sum(nba_players[i]['GP'])), 
                           (sum(nba_players[i]['BLK']) / sum(nba_players[i]['GP']))])
y_train = np.array([0]*5 + [1]*5 + [2]*5)

In [5]:
clf = KNeighborsClassifier()
clf.fit(X_train[:, 1:], y_train)

KNeighborsClassifier()

In [6]:
def predict_position(player_name, clf):    
    player_id = players.find_players_by_full_name(player_name)[0]['id']
    

    player_career = playercareerstats.PlayerCareerStats(player_id=player_id)

    player_df = player_career.get_data_frames()[0]

    player_stats = np.array([(sum(player_df['PTS']) / sum(player_df['GP'])), 
                           (sum(player_df['AST']) / sum(player_df['GP'])), 
                           (sum(player_df['REB']) / sum(player_df['GP'])), 
                           (sum(player_df['STL']) / sum(player_df['GP'])), 
                           (sum(player_df['BLK']) / sum(player_df['GP']))]).reshape(1,-1)
    
    position_id = clf.predict(player_stats)[0]
    position = POSITION_TYPES[position_id]
    return position_id, position

In [7]:
X_test = [
    'michael jordan',
    'shaq',
    'magic johnson',
    'chris paul',
    'marc gasol',
    'dikembe mutombo',
    'russell westbrook',
    'larry bird',
    'john stockton',
    'derrick rose',
    'charles barkley',
    'kyrie irving',
    'rudy gobert',
    'devin booker',
    'steve nash',
    'tim duncan',
    'dirk nowitzki',
    'lebron'
]

y_test = [1, 0, 1, 1, 0, 0, 1, 2, 1, 1, 2, 1, 0, 1, 1, 0, 2, 2]

for nba_star in X_test:
    time.sleep(0.600)
    predicted_position = predict_position(nba_star, clf)[1]
    actual_position = POSITION_TYPES[y_test[X_test.index(nba_star)]]
    print(f'I think {nba_star} is a {predicted_position}. He played {actual_position} in his career')

I think michael jordan is a GUARD. He played GUARD in his career
I think shaq is a CENTER. He played CENTER in his career
I think magic johnson is a GUARD. He played GUARD in his career
I think chris paul is a GUARD. He played GUARD in his career
I think marc gasol is a CENTER. He played CENTER in his career
I think dikembe mutombo is a CENTER. He played CENTER in his career
I think russell westbrook is a GUARD. He played GUARD in his career
I think larry bird is a CENTER. He played FORWARD in his career
I think john stockton is a CENTER. He played GUARD in his career
I think derrick rose is a GUARD. He played GUARD in his career
I think charles barkley is a CENTER. He played FORWARD in his career
I think kyrie irving is a GUARD. He played GUARD in his career
I think rudy gobert is a CENTER. He played CENTER in his career
I think devin booker is a GUARD. He played GUARD in his career
I think steve nash is a CENTER. He played GUARD in his career
I think tim duncan is a CENTER. He played